In [1]:
import sys
sys.path.append('../../')
from taxcalc import *
import numpy as np
import pandas as pd
import copy

In [2]:
dropbox_path = 'C:/Users/cody_/Dropbox/TCJA info/notebook & tax calc code/'
dropbox_path2 = 'C:/Users/cody_/Dropbox/Taxes, Investment & Growth/Model spreadsheets/Version 4/'
growthresults = pd.read_csv(dropbox_path2 + 'growdiffs.csv')
growfact_base = pd.read_csv('../../taxcalc/growfactors.csv')
growfact_ref = copy.deepcopy(growfact_base)
growdiffs = np.concatenate(([0,0,0,0], np.array(growthresults['gfactors'])))
growfact_ref['ATXPY'] = growfact_base['ATXPY'] + growdiffs
growfact_ref['ASCHF'] = growfact_base['ASCHF'] + growdiffs
growfact_ref['ABOOK'] = growfact_base['ABOOK'] + growdiffs
growfact_ref['AWAGE'] = growfact_base['AWAGE'] + growdiffs
growfact_ref['ASCHCI'] = growfact_base['ASCHCI'] + growdiffs
growfact_ref['ASCHEI'] = growfact_base['ASCHEI'] + growdiffs
growfact_ref['AINTS'] = growfact_base['AINTS'] + growdiffs
growfact_ref['ADIVS'] = growfact_base['ADIVS'] + growdiffs
growfact_ref['ACGNS'] = growfact_base['ACGNS'] + growdiffs
growfact_ref['AUCOMP'] = growfact_base['AUCOMP'] + growdiffs
growfact_ref['AIPD'] = growfact_base['AIPD'] + growdiffs
growfact_ref['ATXPY'] = growfact_base['ATXPY'] + growdiffs
growfact_ref['ASCHCL'] = growfact_base['ASCHCL'] - growdiffs
growfact_ref['ASCHEL'] = growfact_base['ASCHEL'] + growdiffs
growfact_ref.to_csv('../../taxcalc/growfactors2.csv', index=False)

In [3]:
def make_calculator2(refdict, year, gfactorname):
    # Creates a calculator advanced to the given year and calculates tax results
    # Note: Passing an empty dictionary to refdict produces a current law calculator
    assert year in range(2014, 2028)
    assert type(refdict) is dict
    growfactor = GrowFactors('../../taxcalc/' + gfactorname)
    pol = Policy()
    beh = Behavior()
    rec = Records('../../puf.csv', gfactors=growfactor)
    if refdict != {}:
        pol.implement_reform(refdict)
    calc1 = Calculator(pol, rec, beh)
    calc1.advance_to_year(year)
    calc1.calc_all()
    return calc1

dropbox_path = 'C:/Users/cody_/Dropbox/TCJA info/notebook & tax calc code/'
# Compile code for distributional analysis
exec(open(dropbox_path + 'distributional_code.py').read())

In [4]:
param = Calculator.read_json_param_objects('../../taxcalc/reforms/2017_law.json', None)
calc_pre = make_calculator2(param['policy'], 2018, 'growfactors.csv')
calc_tcjaS = make_calculator2({}, 2018, 'growfactors.csv')
calc_tcjaD = make_calculator2({}, 2018, 'growfactors2.csv')

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [7]:
# Compile code for equity imputation
exec(open(dropbox_path + 'equity_imputation_code.py').read())
equity = imputeEquity(calc_pre)
dshare = imputeDirectEquity(calc_pre)
wtshare = imputeTaxableIndirectEquity(calc_pre, dshare)
# Compile code for business tax distribution
exec(open(dropbox_path + 'business_distribution_code.py').read())

In [8]:
ranking = {'w_adult': 1, 'w_child': 1, 'elast_size': 0.5}
scaling = {'w_adult': 1, 'w_child': 1, 'elast_size': 0}
excluding = ["neginc"]
screening = ["", ""]
nonprofit_split = {"services": 0.208, "compensation": 0.78, "donors": 0, "foreign": 0.012}
slgov_split = {"benefits": 0.0, "compensation": 1.0, "taxes": 0.0}

In [10]:
static_table1 = applyBtaxDistribution(calc_pre, calc_tcjaS, 2018, equity, dshare, wtshare, nonprofit_split, slgov_split, ranking, scaling, excluding, screening)
dynamic_table1 = applyBtaxDistribution(calc_pre, calc_tcjaD, 2018, equity, dshare, wtshare, nonprofit_split, slgov_split, ranking, scaling, excluding, screening)
combined_table1 = static_table1.merge(dynamic_table1, on="Income group")
combined_table1.to_csv(dropbox_path + 'Growth tables/dynamicdist2018.csv')
combined_table1

Change in corporate tax liability: -94.38
Change in pre-tax income: 75.64332431812663
Change in after-tax income: 242.76805644553144
Change in corporate tax liability: -94.38
Change in pre-tax income: 75.64332431812663
Change in after-tax income: 305.59939962300115


,Income group,Percent_x,Average ($)_x,Percent_y,Average ($)_y
0,Bottom decile,0.005489,14.456917,0.011905,31.354474
1,Second decile,0.008092,99.208834,0.013230,162.196704
2,Third decile,0.015726,292.125087,0.019986,371.274890
3,Fourth decile,0.018105,455.697759,0.022155,557.615329
4,Fifth decile,0.016521,549.172586,0.021053,699.805469
5,Sixth decile,0.017354,756.216024,0.022412,976.603956
6,Seventh decile,0.019116,1069.300692,0.024512,1371.169123
7,Eighth decile,0.021172,1554.070247,0.026733,1962.189941
8,Ninth decile,0.022111,2234.268361,0.027895,2818.722251
9,Next 5%,0.023748,3388.831012,0.029853,4260.028887


In [11]:
static_table2 = applyBtaxDistribution(calc_pre, calc_tcjaS, 2023, equity, dshare, wtshare, nonprofit_split, slgov_split, ranking, scaling, excluding, screening)
dynamic_table2 = applyBtaxDistribution(calc_pre, calc_tcjaD, 2023, equity, dshare, wtshare, nonprofit_split, slgov_split, ranking, scaling, excluding, screening)
combined_table2 = static_table2.merge(dynamic_table2, on="Income group")
combined_table2.to_csv(dropbox_path + 'Growth tables/dynamicdist2023.csv')
combined_table2

Change in corporate tax liability: -7.383
Change in pre-tax income: 5.9172988285724735
Change in after-tax income: 197.79783696520087
Change in corporate tax liability: -7.383
Change in pre-tax income: 5.9172988285724735
Change in after-tax income: 403.6916928624274


,Income group,Percent_x,Average ($)_x,Percent_y,Average ($)_y
0,Bottom decile,0.002902,9.293718,0.020012,64.084943
1,Second decile,0.008226,117.700995,0.021752,311.233478
2,Third decile,0.014320,304.670842,0.025367,539.720962
3,Fourth decile,0.012651,368.179284,0.023077,671.634376
4,Fifth decile,0.011665,446.042895,0.023774,909.087851
5,Sixth decile,0.012729,638.436892,0.026063,1307.276693
6,Seventh decile,0.014792,947.046386,0.029095,1862.735052
7,Eighth decile,0.016568,1391.094206,0.031495,2644.342105
8,Ninth decile,0.015062,1744.594264,0.030706,3556.613527
9,Next 5%,0.015154,2462.311067,0.031669,5145.838663


In [12]:
static_table3 = applyBtaxDistribution(calc_pre, calc_tcjaS, 2027, equity, dshare, wtshare, nonprofit_split, slgov_split, ranking, scaling, excluding, screening)
dynamic_table3 = applyBtaxDistribution(calc_pre, calc_tcjaD, 2027, equity, dshare, wtshare, nonprofit_split, slgov_split, ranking, scaling, excluding, screening)
combined_table3 = static_table3.merge(dynamic_table3, on="Income group")
combined_table3.to_csv(dropbox_path + 'Growth tables/dynamicdist2027.csv')
combined_table3

Change in corporate tax liability: -57.566
Change in pre-tax income: 46.13777927206695
Change in after-tax income: 4.098664828378244
Change in corporate tax liability: -57.566
Change in pre-tax income: 46.13777927206695
Change in after-tax income: 195.59576213398861


,Income group,Percent_x,Average ($)_x,Percent_y,Average ($)_y
0,Bottom decile,0.000966,3.566626,0.014690,54.264462
1,Second decile,-0.002354,-38.242968,0.007842,127.427624
2,Third decile,-0.002961,-70.928089,0.005339,127.897321
3,Fourth decile,-0.002026,-66.508051,0.006089,199.875072
4,Fifth decile,-0.000672,-28.985934,0.008765,377.883277
5,Sixth decile,-0.000193,-10.919895,0.010051,568.621376
6,Seventh decile,-0.000305,-21.924062,0.010731,771.693310
7,Eighth decile,-0.000734,-69.329100,0.011004,1038.930409
8,Ninth decile,-0.000553,-72.200780,0.011930,1556.464045
9,Next 5%,0.000120,21.869372,0.013004,2378.663471
